Clean up our environment

In [ ]:
USE master;
GO
DECLARE @dbname NVARCHAR(128) = N'SSDTWithtSQLt';

IF (
    EXISTS(
        SELECT name
        FROM master.dbo.sysdatabases
        WHERE ('[' + name + ']' = @dbname OR name = @dbname)
    )
)
BEGIN
    ALTER DATABASE [SSDTWithtSQLt] SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
END
GO
DROP DATABASE IF EXISTS SSDTWithtSQLt;
GO
-- Cleanup
DROP PROCEDURE IF EXISTS [dbo].[Vessel_Create];
DROP PROCEDURE IF EXISTS [dbo].[Vessel_GetAll];
GO

Create the database

In [ ]:
CREATE DATABASE SSDTWithtSQLt;
GO

Create the tables

In [ ]:
USE [SSDTWithtSQLt];
GO
DROP TABLE IF EXISTS dbo.Operator
CREATE TABLE [dbo].[Operator]
(
    [OperatorID] [INT] IDENTITY(1, 1) NOT NULL,
    [OperatorCode] CHAR(3) NOT NULL,
    [OperatorName] VARCHAR(20) NOT NULL,
    CONSTRAINT [PK_Operator]
        PRIMARY KEY CLUSTERED ([OperatorID] ASC)
        WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON,ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]
) ON [PRIMARY];
GO
DROP TABLE IF EXISTS dbo.Vessel
CREATE TABLE [dbo].[Vessel]
(
    [VesselID] [INT] IDENTITY(1, 1) NOT NULL,
    [VesselCode] CHAR(3) NOT NULL,
    [VesselName] [VARCHAR](30) NULL,
    [TEU] int NULL,
    [Plug] int NULL,
    [OperatorID] INT NULL,
    CONSTRAINT [PK_Vessel]
        PRIMARY KEY CLUSTERED ([VesselID] ASC)
        WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON,ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]
) ON [PRIMARY];
GO
ALTER TABLE [dbo].[Vessel] WITH CHECK
ADD CONSTRAINT [FK_Vessel_Operator]
    FOREIGN KEY ([OperatorID])
    REFERENCES [dbo].[Operator] ([OperatorID]);
GO
ALTER TABLE [dbo].[Vessel] CHECK CONSTRAINT [FK_Vessel_Operator];
GO

Create the procedures

In [ ]:
USE [SSDTWithtSQLt];
GO
DROP PROC IF EXISTS [dbo].[Vessel_Create]
GO
CREATE PROCEDURE [dbo].[Vessel_Create]
    @VesselID INT OUTPUT,
    @VesselCode CHAR(3) = NULL, --it does not make sense but it's for demo purpose
    @VesselName VARCHAR(30) = NULL,
    @TEU int = NULL,
    @Plug int = NULL,
    @OperatorID INT = NULL
AS
/*
<documentation>
  <author>ABM040</author>
  <summary>Create Operator used on Vessels</summary>
  <returns>
            OperatorID created
   </returns>
</documentation>

Changes:
Date		Who						Notes
----------	---						--------------------------------------------------------------
20210714	ABM040				    Creation
*/


    -- Set session options to make sure transactions are aborted correctly
    -- and the procedure doesn't return the count
    SET XACT_ABORT, NOCOUNT ON;

    -- Check the parameters
    IF (@VesselCode IS NULL)
    BEGIN
        ;
        THROW 50000, 'Invalid parameter: @VesselCode cannot be NULL!', 1;
        RETURN;
    END;

    INSERT INTO dbo.Vessel
    (
        -- VesselID -- this column value is auto-generated
        VesselCode,
        VesselName,
        TEU,
        Plug,
        OperatorID
    )
    VALUES
    (
        @VesselCode,
        @VesselName,
        @TEU,
        @Plug,
        @OperatorID
    )

    SELECT @VesselID = SCOPE_IDENTITY()
GO
DROP PROC IF EXISTS [dbo].[Vessel_GetAll]
GO
CREATE PROCEDURE [dbo].[Vessel_GetAll]
AS
/*
<documentation>
  <author>ABM040</author>
  <summary>Get List of Vessels</summary>
  <returns>
            VesselID, VesselCode, VesselName, TEU, Plug, OperatorID, OperatorCode
  </returns>
</documentation>

Changes:
Date		Who						Notes
----------	---						--------------------------------------------------------------
20210714	ABM040				    Creation
*/
SET NOCOUNT ON;

SELECT v.VesselID
      ,v.VesselCode
      ,v.VesselName
      ,v.TEU
      ,v.Plug
      ,v.OperatorID
      ,o.OperatorCode
FROM [dbo].[Vessel] v 
LEFT JOIN [dbo].[Operator] o on o.OperatorID = v.OperatorID
GO